In [ ]:
import pandas as pd
from pyexpat import features

data = pd.read_parquet('../cache/prepd_99q.parquet')
data = data.resample('15min').ffill()
data.head()

In [ ]:
data.dtypes

In [ ]:
# Split the data into train and test sets
train_data = data[data.index.year < 2023]
test_data = data[data.index.year == 2023]

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)
smol_sample = train_data.tail(1000)
sample = train_data.tail(10000)
big_sample = train_data.tail(100000)
sample.head()

In [ ]:
# cache the train and test data for later use
train_data.to_parquet('../cache/train_data.parquet')
test_data.to_parquet('../cache/test_data.parquet')

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Quantifies multicollinearity
vif_data = pd.DataFrame()
vif_data["Variable"] = sample.columns
vif_data["VIF"] = [variance_inflation_factor(sample.values, i) for i in range(sample.shape[1])]
print(vif_data)

### Vector Autoregression (VAR)

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

model = VAR(big_sample)
var_results = model.fit(maxlags=15, ic='aic') # Fit with automatic lag order selection based on AIC
lag_order = var_results.k_ar
predictions = var_results.forecast(sample.values[-lag_order:], steps=5) # Forecast 5 steps ahead
print(predictions)

### dynamic factor model (DFM)

In [ ]:
from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor

# Fit a dynamic factor model
model = DynamicFactor(sample, k_factors=1, factor_order=1)
dfm_results = model.fit()
print(dfm_results.summary())

In [ ]:
forecast_steps = 5
forecast = dfm_results.get_forecast(steps=forecast_steps)
# Extract the predicted values from the forecast result
predicted_values = forecast.predicted_mean
# Or for the full prediction including uncertainty (confidence intervals)
prediction_conf_int = forecast.conf_int()

# Display the predicted values and the confidence intervals
print("Predicted Values:")
print(predicted_values)

print("\nConfidence Intervals for Predictions:")
print(prediction_conf_int)

### Vector Autoregression Moving-Average (VARMA)

In [ ]:
from statsmodels.tsa.statespace.varmax import VARMAX

model = VARMAX(smol_sample, order=(1, 1))  # (p, q) - autoregressive and moving average orders
varmax_results = model.fit(disp=False)
predictions = varmax_results.forecast(steps=5)
print(predictions)

In [ ]:
# cache the models for later use
import joblib
joblib.dump(var_results, '../cache/var_model.joblib')
joblib.dump(dfm_results, '../cache/dfm_model.joblib')
joblib.dump(varmax_results, '../cache/varmax_model.joblib')

In [ ]:
# purely for testing
# Load the models from cache
var_results_loaded = joblib.load('../cache/var_model.joblib')
print("done")
var_results_loaded.forecast(sample.values[var_results_loaded.k_ar:], steps=5)